![](https://wherobots.com/wp-content/uploads/2023/12/Inline-Blue_Black_onWhite@3x.png)

# Introduction to `VTiles` for WherobotsDB

In this notebook we will create PMTiles vector tiles for rendering maps using building and road data from the Overture Maps dataset in the Wherobots Open Data Catalog.

## Introduction To Vector Tiles

Vector tiles provide performant rendering of map data for large vector feature datasets across large regions and zoom
levels. Here’s why, and when, they should be used:

* Vector tiles are designed for use in web maps, mobile apps, and desktop GIS software.
* WherobotsDB makes it easy and affordable to generate vector tiles at a planetary scale.
* By rendering vector tiles directly, the interactive map experience is more responsive and scalable for large datasets
  than rendering feature formats (e.g., GeoJSON) directly and allows developers to customize the display, which is
  otherwise impossible with raster tiles.

## How To Generate Vector Tiles

### Start a Sedona Session

As always, begin by starting a Sedona context

In [1]:
from sedona.spark import *

config = SedonaContext.builder().getOrCreate()
sedona = SedonaContext.create(config)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Load Feature Data

Create a Spatial DataFrame with a geometry column and a layer column. The geometry column contains the features to render in the
map. The layer column is a string that describes the grouping the feature should be in. Records within the same layer
can be styled together, independently of other layers. In this case example features that represent buildings are in the buildings layer and those representing roads are in the roads layer.

The first cell that follows gives some variable to control where we generate tiles for. The default is a small town in Washington: Issaquah.

In [5]:
from sedona.sql.st_constructors import ST_GeomFromText
from sedona.sql.st_predicates import ST_Intersects
import pyspark.sql.functions as f

# Set to False to generate tiles for the entire dataset, True to generate only for region_wkt area
filter = True
region_wkt = "POLYGON ((-122.097931 47.538528, -122.048836 47.566566, -121.981888 47.510012, -122.057076 47.506302, -122.097931 47.538528))"
filter_expression = ST_Intersects(f.col("geometry"), ST_GeomFromText(f.lit(region_wkt)))

Next, we create the buildings Spatial DataFrame using the Overture buildings table from the Wherobots Open Data Catalog.

In [6]:
import pyspark.sql.functions as f

buildings_df = (
    sedona.table("wherobots_open_data.overture_2024_02_15.buildings_building")
    .select(
        f.col("geometry"),
        f.lit("buildings").alias("layer"),
        f.element_at(f.col("sources"), 1).dataset.alias("source")
    )
)

buildings_df.show()

+--------------------+---------+--------------------+
|            geometry|    layer|              source|
+--------------------+---------+--------------------+
|POLYGON ((34.6701...|buildings|Microsoft ML Buil...|
|POLYGON ((34.6842...|buildings|Microsoft ML Buil...|
|POLYGON ((34.7023...|buildings|Google Open Build...|
|POLYGON ((34.7025...|buildings|Microsoft ML Buil...|
|POLYGON ((34.7018...|buildings|Microsoft ML Buil...|
|POLYGON ((34.6989...|buildings|Microsoft ML Buil...|
|POLYGON ((34.6990...|buildings|Google Open Build...|
|POLYGON ((34.6990...|buildings|Google Open Build...|
|POLYGON ((34.7067...|buildings|Microsoft ML Buil...|
|POLYGON ((34.7163...|buildings|Google Open Build...|
|POLYGON ((34.7125...|buildings|Google Open Build...|
|POLYGON ((34.7081...|buildings|Microsoft ML Buil...|
|POLYGON ((34.7010...|buildings|Microsoft ML Buil...|
|POLYGON ((34.7007...|buildings|Microsoft ML Buil...|
|POLYGON ((34.7011...|buildings|Microsoft ML Buil...|
|POLYGON ((34.7031...|buildi

Next, we create a Spatial DataFrame for our road features using the Overture transportation segment table.

In [7]:
roads_df = (
    sedona.table("wherobots_open_data.overture_2024_02_15.transportation_segment")
    .select(
        f.col("geometry"),
        f.lit("roads").alias("layer"),
        f.element_at(f.col("sources"), 1).dataset.alias("source")
    )
)

roads_df.show()

+--------------------+-----+-------------+
|            geometry|layer|       source|
+--------------------+-----+-------------+
|LINESTRING (1.407...|roads|OpenStreetMap|
|LINESTRING (1.407...|roads|OpenStreetMap|
|LINESTRING (1.411...|roads|OpenStreetMap|
|LINESTRING (1.412...|roads|OpenStreetMap|
|LINESTRING (1.411...|roads|OpenStreetMap|
|LINESTRING (1.413...|roads|OpenStreetMap|
|LINESTRING (1.413...|roads|OpenStreetMap|
|LINESTRING (1.413...|roads|OpenStreetMap|
|LINESTRING (1.411...|roads|OpenStreetMap|
|LINESTRING (1.414...|roads|OpenStreetMap|
|LINESTRING (1.413...|roads|OpenStreetMap|
|LINESTRING (1.418...|roads|OpenStreetMap|
|LINESTRING (1.419...|roads|OpenStreetMap|
|LINESTRING (1.430...|roads|OpenStreetMap|
|LINESTRING (1.431...|roads|OpenStreetMap|
|LINESTRING (1.436...|roads|OpenStreetMap|
|LINESTRING (1.430...|roads|OpenStreetMap|
|LINESTRING (1.451...|roads|OpenStreetMap|
|LINESTRING (1.455...|roads|OpenStreetMap|
|LINESTRING (1.459...|roads|OpenStreetMap|
+----------

Next, we prepare a single spatial DataFrame combining our roads and buildings features.

In [8]:
features_df = roads_df.union(buildings_df)

if filter: 
    features_df = features_df.filter(ST_Intersects(f.col("geometry"), ST_GeomFromText(f.lit(region_wkt))))

features_df.count()


8718

### Create Tiles as a PMTiles Archive

Once we have the Spatial DataFrame ready for tile generation, we can use the `vtiles.generate_pmtiles` method to create a PMTiles archive. PMTiles is a performant, simple, and optimized format for storing vector tiles.

Wherobots will automatically handle the details for you. However, if you need more control, a `GenerationConfig` object can optionally be provided as an argument to control which tiles are created and their contents. A `PMTilesConfig` object can optionally be provided to control the header information of the PMTiles Archive.



In [9]:
from wherobots import vtiles
import os

full_tiles_path = os.getenv("USER_S3_PATH") + "tiles.pmtiles"
vtiles.generate_pmtiles(features_df, full_tiles_path)


In [ ]:

vtiles.write_pmtiles(tiles_df, full_tiles_path, features_df=features_df)

## Visualizing Vector Tiles with leafmap

We’ve made it easy to use [leafmap](https://leafmap.org/) to visualize the vector tiles we just generated. Leafmap
is a popular geospatial visualization tool for Jupyter notebooks. We offer a function that makes it easy to use leafmap.
This function creates a signed URL, styles the tiles, and returns a Leafmap object. The function can be used as follows:


In [10]:
vtiles.show_pmtiles(full_tiles_path)

## Quick Generation of Tiles

Sometimes you want to quickly visualize a massive dataset. To achieve this goal, WherobotsDB provides a function for
quickly generating, saving, and displaying tiles. When testing this function it completed 100 million features in
less than 5 minutes on a Wherobots Cloud Cairo runtime. This is accomplished by limiting the features processed to 100 million and
generating fewer zoom levels at a higher resolution. At high zooms, the low precision from the low maximum zoom may be
evident.

The Scala/Java API exposes the `getQuickConfig` method which provides the same `GenerationConfig`. This can be passed to
the `vtiles.generate` or `vtiles.generatePMTiles` methods for the same tile generation functionality.

This feature can be used as follows:

In [ ]:
sample_tiles_path = os.getenv("USER_S3_PATH") + "sampleTiles.pmtiles"
vtiles.generate_quick_pmtiles(features_df, sample_tiles_path)

25/01/10 04:55:57 WARN CacheManager: Asked to cache already cached data.        


As a comprehensive map application toolbox, WherobotsDB provides many off-the-shelf scalable tools. In this
tutorial, we just focus on a minimum example. Detailed explanation of each tool can be found
in [the documentation](https://docs.wherobots.com/latest/references/wherobotsdb/vector-data/Overview/).